In [1]:
# Automatically reload changes to external code
%load_ext autoreload
%autoreload 2


#A Convolutional Network implementation example using TensorFlow library.
#This example is using the MNIST database of handwritten digits
#(http://yann.lecun.com/exdb/mnist/)

#Author: Aymeric Damien
#Project: https://github.com/aymericdamien/TensorFlow-Examples/

# Based on above project, modified by James Chan

In [2]:
import tensorflow as tf
from mnist import loader #loader for mnist dataset
from matplotlib import pyplot as plt
import numpy as np
import pdb, time, cv2

# Mnist dataset

The MNIST database of handwritten digits. [[website]](http://yann.lecun.com/exdb/mnist/)<br>
There are **60,000** training images and **10,000** testing images in this dataset.<br>
Each digit is a one-channel image. Size of image = 28*28 = 784.

![](imgs/mnist_ex.png)

There are some build-in mnist function can be used in tensorflow.

Ex.<br>
from tensorflow.examples.tutorials.mnist import input_data<br>
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instead of using these functions, I'll use the orginal dataset manually in this code.<br>
It's more clear to trace the data-processing.

When we load the data


In [3]:
# load mnist data manually
# loading 'train' or 'test' data
# ex. load_mnist_data('train')
# return images, labels and mean of all images. (But, we'll only use the mean of training data.)
# ims: [N * 784]
# labels: [N]
# ims_mean: [784]

def load_mnist_data(flag, data_path='data'):
    data_loader = loader.MNIST(data_path)
    if flag == 'train':
        ims, labels = data_loader.load_training()
    elif flag == 'test':
        ims, labels = data_loader.load_testing()
    else:
        raise ValueError("Error. Only training or testing data.")
    ims = ims/255.0
    ims_mean = np.mean(ims, axis=0)
    return ims, labels, ims_mean

In [21]:
# Parameters
learning_rate = 0.001  # default = 0.001
training_epochs = 2 # default = 2
batch_size = 50   # training batch size, default = 50
test_batch_size = 100
display_step = 50  # testing, default = 50

# Network Parameters
n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
stddev=0.01    # standard deviation for random initialization


# Functions of Convolutional Neural Network

In [5]:
# Create some wrappers for simplicity

# convolutional function
# input: 
# x=[batch_size, height, width, channels]
# W(Weights)=tf.Variable(shape=[kernel_size, kernel_size, input_channel, output_channel])
# b(Biases)=tf.Variable(shape=[output_channel])
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')


In [6]:
def lenet():
    # tf Graph input
    x = tf.placeholder(tf.float32, [None, n_input])  # mnist input images, [batch_size x 784]
    y = tf.placeholder(tf.int32,[None])              # label, [batch_size]
    dropout = tf.placeholder(tf.float32)  #dropout ratio
    
    # Declare Variables 
    # Store layers weight & bias
    weights = {
        # 5x5 conv, 1 input, 32 outputs
        'wc1': tf.Variable(tf.truncated_normal([5, 5, 1, 32], mean=0, stddev=stddev)),
        # 5x5 conv, 32 inputs, 64 outputs
        'wc2': tf.Variable(tf.truncated_normal([5, 5, 32, 64], mean=0, stddev=stddev)),
        # fully connected, 7*7*64 inputs, 1024 outputs
        'wd1': tf.Variable(tf.truncated_normal([7*7*64, 1024], mean=0, stddev=stddev)),
        # 1024 inputs, 10 outputs (class prediction)
        'out': tf.Variable(tf.truncated_normal([1024, n_classes], mean=0, stddev=stddev))
    }

    biases = {
        'bc1': tf.Variable(tf.random_normal([32])),  # default stddev=1.0
        'bc2': tf.Variable(tf.random_normal([64])),
        'bd1': tf.Variable(tf.random_normal([1024])),
        'out': tf.Variable(tf.random_normal([n_classes]))
    }

    # Construct model
    x_reshape = tf.reshape(x, shape=[-1, 28, 28, 1]) # Transfer shape. Prepare for convolution

    # Convolution Layer
    conv1 = conv2d(x_reshape, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, keep_prob = 1-dropout)   # dropout ratio --> keep ratio

    # Output, class prediction
    pred = tf.add(tf.matmul(fc1, weights['out']), biases['out'])

    # Define loss and optimizer
    #one_hot_y = tf.one_hot(y, n_classes, on_value=1, off_value=0, axis=-1)
    #one_hot_y = tf.cast(one_hot_y, tf.float32)
    #cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=one_hot_y))

    

    probs = tf.nn.softmax(pred)
    log_probs = tf.log(probs + 1e-8)

    one_hot_y = tf.one_hot(y, n_classes, on_value=1, off_value=0, axis=-1)
    #print one_hot_y.get_shape()
    #cross_entropy_loss = - tf.mul(y,log_probs)
    cross_entropy_loss = - tf.mul(tf.cast(one_hot_y, tf.float32),log_probs)
    
    loss = tf.reduce_sum(cross_entropy_loss)
    
    # Evaluate model
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(one_hot_y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


    return x, y, dropout, loss, pred, accuracy, conv1, conv2


In [7]:
def eval_model(sess, x, y, dropout, ims, labels, ims_test, labels_test, ims_mean, iter_per_epoch, test_iter):
    Train_Loss = 0
    Test_Loss = 0
    Train_Acc = 0
    Test_Acc = 0
    for idx in xrange(iter_per_epoch):
        batch_xs = ims[order_list[idx*batch_size:(idx+1)*batch_size]] - ims_mean
        batch_ys = labels[order_list[idx*batch_size:(idx+1)*batch_size]]
        C, A = sess.run([cost, accuracy], feed_dict={x: batch_xs, y: batch_ys, dropout: 0.0})
        Train_Loss += C/batch_size   # calculate the loss in average (per image).
        Train_Acc += A
    # Eval testing dataset
    for idx in xrange(test_iter):
        batch_xs = ims_test[order_list[idx*test_batch_size:(idx+1)*test_batch_size]] - ims_mean
        batch_ys = labels_test[order_list[idx*test_batch_size:(idx+1)*test_batch_size]]
        C, A = sess.run([cost, accuracy], feed_dict={x: batch_xs, y: batch_ys, dropout: 0.0})
        Test_Loss += C/test_batch_size
        Test_Acc += A
    return Train_Loss, Train_Acc, Test_Loss, Test_Acc

In [8]:
def plot_fcn(types, tag='train'):
    # dropout=None, lr=None, batchsize=None, normalize=None
    npzfiles=[]
    colors = ['b-', 'r-', 'k-', 'g-', 'y-', 'c-', 'm-']
    if len(types) > len(colors):
        print "only accept %d types"%(len(color))
        return 
    
    for i in xrange(len(types)):
        npz = np.load('npzfiles/'+types[i]+'.npz')
        npzfiles.append(npz)
    print npzfiles
    plt.figure(1)
    plt.subplot(211)
    plt.title('Loss.')
    plt.xlabel('Epochs')
    plt.axis([0, training_epochs, 0, 5.0])
    for i in xrange(len(npzfiles)):
        plt.plot(npzfiles[i]['loss'].item()['x'], npzfiles[i]['loss'].item()[tag+'_y'], colors[i], label=types[i])
    plt.legend(loc='upper left')
    plt.subplot(212)
    plt.title('Accuracy.')
    plt.xlabel('Epochs')
    plt.ylabel('(%)')
    plt.axis([0, training_epochs, 0, 1.0])
    for i in xrange(len(npzfiles)):
        plt.plot(npzfiles[i]['acc'].item()['x'], npzfiles[i]['acc'].item()[tag+'_y'], colors[i], label=types[i])
    plt.show()

# Main function



In [22]:
# loading training and testing data
ims, labels, ims_mean = load_mnist_data('train', data_path='data')
#ims_mean = np.zeros((28*28))
ims_test, labels_test, _ = load_mnist_data('test', data_path='data')

order_list = range(len(ims))

# parameters related to mnist dataset 
test_iter = len(ims_test)/test_batch_size # number of testing-minibatch.

iter_per_epoch = len(ims)/batch_size      # number of training-minibatch.

# Launch the graph
with tf.Session() as sess:

    # Keep training until reach max iterations
    x, y, dropout, cost, pred, accuracy, conv1, conv2= lenet()
    train_loss = cost/batch_size # loss per image
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9).minimize(train_loss)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(train_loss)



    # initialize all variables
    try:
        init = tf.initialize_all_variables()
    except:
        init = tf.global_variables_initializer()
    sess.run(init)
    Loss_plt = {'x':[], 'train_y':[], 'test_y':[]}
    Acc_plt  = {'x':[], 'train_y':[], 'test_y':[]} 
    # Before Training (Random initialization), Evaluate the model one-time.
    begin = time.time()
    Train_Loss, Train_Acc, Test_Loss, Test_Acc=eval_model(sess, x, y, dropout, ims, labels, ims_test, labels_test, ims_mean, iter_per_epoch, test_iter)
    print "------After Random Initialization------"
    print "Training: loss=%f, acc=%f.\t\tTesting: loss=%f, acc=%f" %(Train_Loss/iter_per_epoch, Train_Acc/iter_per_epoch,
                                                                     Test_Loss/test_iter, Test_Acc/test_iter)
    
    epoch = 0
    step = 1      
    
    Loss_plt['x'].append(float(step)/iter_per_epoch)
    Loss_plt['train_y'].append(Train_Loss/iter_per_epoch)
    Loss_plt['test_y'].append(Test_Loss/test_iter)
    Acc_plt['x'].append(float(step)/iter_per_epoch)
    Acc_plt['train_y'].append(Train_Acc/iter_per_epoch)
    Acc_plt['test_y'].append(Test_Acc/test_iter)
    
    duration = time.time()-begin
    print " %f seconds"%(duration)
    
    print "------Start Training------"

    for epoch in xrange(training_epochs):
        begin = time.time()
        Train_Loss = 0
        Test_Loss = 0
        Train_Acc = 0
        Test_Acc = 0
        for idx in xrange(iter_per_epoch):
            batch_xs = ims[order_list[idx*batch_size:(idx+1)*batch_size]] - ims_mean
            batch_ys = labels[order_list[idx*batch_size:(idx+1)*batch_size]]
            # Run optimization op (backprop)
            #print batch_ys.shape
            sess.run([optimizer], feed_dict={x: batch_xs, y: batch_ys, dropout: 0.5})
            if step % display_step == 0:
                Train_Loss, Train_Acc, Test_Loss, Test_Acc=eval_model(sess, x, y, dropout, ims, labels, ims_test, labels_test, ims_mean, iter_per_epoch, test_iter)
                print "Epoch %f, Training: loss=%f, acc=%f.\t\tTesting: loss=%f, acc=%f"%(float(step)/iter_per_epoch, Train_Loss/iter_per_epoch, Train_Acc/iter_per_epoch, 
                                                                                         Test_Loss/test_iter, Test_Acc/test_iter)
                Loss_plt['x'].append(float(step)/iter_per_epoch)
                Loss_plt['train_y'].append(Train_Loss/iter_per_epoch)
                Loss_plt['test_y'].append(Test_Loss/test_iter)
                Acc_plt['x'].append(float(step)/iter_per_epoch)
                Acc_plt['train_y'].append(Train_Acc/iter_per_epoch)
                Acc_plt['test_y'].append(Test_Acc/test_iter)
            step += 1
        
        # Evaluate after each epoch finished.
        Train_Loss, Train_Acc, Test_Loss, Test_Acc=eval_model(sess, x, y, dropout, ims, labels, ims_test, labels_test, ims_mean, iter_per_epoch, test_iter)
        print "Epoch %d, Training: loss=%f, acc=%f.\t\tTesting: loss=%f, acc=%f"%(epoch+1, Train_Loss/iter_per_epoch, Train_Acc/iter_per_epoch, 
                                                                                         Test_Loss/test_iter, Test_Acc/test_iter)
        Loss_plt['x'].append(float(step)/iter_per_epoch)
        Loss_plt['train_y'].append(Train_Loss/iter_per_epoch)
        Loss_plt['test_y'].append(Test_Loss/test_iter)
        Acc_plt['x'].append(float(step)/iter_per_epoch)
        Acc_plt['train_y'].append(Train_Acc/iter_per_epoch)
        Acc_plt['test_y'].append(Test_Acc/test_iter)

        duration = time.time()-begin
        print "Cost %f seconds"%(duration)
        
        np.savez('outfile', loss=Loss_plt, acc=Acc_plt)
        Vis=False
        # Vis
        if Vis == True:
            index = np.random.randint(len(ims))
            batch_ys = np.zeros((1))
            batch_ys[0] = labels[index]
            C1, C2 = sess.run([conv1, conv2], feed_dict={x: ims[index][np.newaxis,:], y:batch_ys, dropout: 0.0})
            print C1.shape, C2.shape
            for i in xrange(32):
                plt.subplot(10,10,i+1),plt.imshow(C1[0,:,:,i],cmap='Greys_r'),plt.title('CONV1')
            for i in xrange(64):
                plt.subplot(10,10,100-i),plt.imshow(C2[0,:,:,i],cmap='Greys_r'),plt.title('CONV2')
            plt.show()                                 
    print("Optimization Finished!")
    
    plt.figure(1)
    plt.subplot(211)
    plt.title('Loss.')
    plt.xlabel('Epochs')
    plt.axis([0, training_epochs, 0, 5.0])
    plt.plot(Loss_plt['x'], Loss_plt['train_y'], 'b-', label='training')
    plt.plot(Loss_plt['x'], Loss_plt['test_y'], 'r-', label='testing')
    plt.legend(loc='upper left')
    plt.subplot(212)
    plt.title('Accuracy.')
    plt.xlabel('Epochs')
    plt.ylabel('(%)')
    plt.axis([0, training_epochs, 0, 1.0])
    plt.plot(Acc_plt['x'], Acc_plt['train_y'], 'b-', label='training')
    plt.plot(Acc_plt['x'], Acc_plt['test_y'], 'r-', label='testing')
    plt.legend(loc='upper left')
    plt.show()
 

------After Random Initialization------
Training: loss=2.491240, acc=0.102183.		Testing: loss=2.492461, acc=0.101000
 5.585631 seconds
------Start Training------
Epoch 0.041667, Training: loss=2.322421, acc=0.097367.		Testing: loss=2.323209, acc=0.098200
Epoch 0.083333, Training: loss=2.208141, acc=0.236800.		Testing: loss=2.206677, acc=0.231300
Epoch 0.125000, Training: loss=1.113347, acc=0.682750.		Testing: loss=1.099100, acc=0.689600
Epoch 0.166667, Training: loss=0.824046, acc=0.709617.		Testing: loss=0.796710, acc=0.720100
Epoch 0.208333, Training: loss=0.640204, acc=0.789117.		Testing: loss=0.621286, acc=0.801500
Epoch 0.250000, Training: loss=0.548887, acc=0.822117.		Testing: loss=0.521455, acc=0.835500
Epoch 0.291667, Training: loss=0.442021, acc=0.857767.		Testing: loss=0.422776, acc=0.868800
Epoch 0.333333, Training: loss=0.405092, acc=0.869650.		Testing: loss=0.378290, acc=0.881400
Epoch 0.375000, Training: loss=0.375293, acc=0.880250.		Testing: loss=0.353931, acc=0.889500
E

In [20]:
# See results
types=['batchsize5', 'batchsize50', 'batchsize500']
plot_fcn(types, tag='train')

IOError: [Errno 2] No such file or directory: 'npzfiles/batchsize50.npz'

# Possible Results 

------After Random Initialization------<br>
Training: loss=2.387872, acc=0.104417.		Testing: loss=2.384974, acc=0.102800<br>
 1.819188 seconds<br>
------Start Training------<br>
Epoch 0.000000, Training: loss=3.225684, acc=0.098717.		Testing: loss=3.228740, acc=0.098000<br>
Epoch 0.083333, Training: loss=2.297542, acc=0.125050.		Testing: loss=2.298005, acc=0.127100<br>
Epoch 0.166667, Training: loss=1.328221, acc=0.569167.		Testing: loss=1.308034, acc=0.587500<br>
Epoch 0.250000, Training: loss=0.686311, acc=0.780217.		Testing: loss=0.664026, acc=0.796700<br>
Epoch 0.333333, Training: loss=0.498580, acc=0.836733.		Testing: loss=0.478692, acc=0.843300<br>
Epoch 0.416667, Training: loss=0.363654, acc=0.888833.		Testing: loss=0.351688, acc=0.892700<br>
Epoch 0.500000, Training: loss=0.331383, acc=0.907383.		Testing: loss=0.321058, acc=0.906400<br>
Epoch 0.583333, Training: loss=0.252823, acc=0.924283.		Testing: loss=0.241469, acc=0.925900<br>
Epoch 0.666667, Training: loss=0.261203, acc=0.916517.		Testing: loss=0.252525, acc=0.918900<br>
Epoch 0.750000, Training: loss=0.212862, acc=0.932667.		Testing: loss=0.203338, acc=0.936300<br>
Epoch 0.833333, Training: loss=0.192696, acc=0.942000.		Testing: loss=0.185617, acc=0.941900<br>
Epoch 0.916667, Training: loss=0.176930, acc=0.945600.		Testing: loss=0.169035, acc=0.946700<br>
Epoch 1, Training: loss=0.185506, acc=0.941167.		Testing: loss=0.174071, acc=0.945100<br>
Cost 27.885424 seconds<br>
Optimization Finished!<br>


![](imgs/result_all.png)

# Tensorflow python API

### tf.nn.conv2d(input, filter, strides, padding, use_cudnn_on_gpu=None, data_format=None, name=None)

Computes a 2-D convolution given 4-D input and filter tensors.

Given an **input tensor of shape [batch, in_height, in_width, in_channels]** and a **filter / kernel tensor of shape [filter_height, filter_width, in_channels, out_channels]**, this op performs the following:

Flattens the filter to a 2-D matrix with shape [filter_height * filter_width * in_channels, output_channels].

Extracts image patches from the input tensor to form a virtual tensor of shape [batch, out_height, out_width, filter_height * filter_width * in_channels].
For each patch, right-multiplies the filter matrix and the image patch vector.

In detail, with the default NHWC format,

output[b, i, j, k] = sum_{di, dj, q} input[b, strides[1] * i + di, strides[2] * j + dj, q] * filter[di, dj, q, k]


Must have strides[0] = strides[3] = 1. For the most common case of the same horizontal and vertices strides, strides = [1, stride, stride, 1].

**Args:**

> input: A Tensor. Must be one of the following types: half, float32, float64.

> filter: A Tensor. Must have the same type as input.

> strides: A list of ints. 1-D of length 4. The stride of the sliding window for each dimension of input. Must be in the same order as the dimension specified with format.

> padding: A string from: "SAME", "VALID". The type of padding algorithm to use.
use_cudnn_on_gpu: An optional bool. Defaults to True.

> data_format: An optional string from: "NHWC", "NCHW". Defaults to "NHWC". Specify the data format of the input and output data. With the default format "NHWC", the data is stored in the order of: [batch, in_height, in_width,
in_channels]. Alternatively, the format could be "NCHW", the data storage order of: [batch, in_channels, in_height, in_width].

> name: A name for the operation (optional).


**Returns:**

> A Tensor. Has the same type as input.
